# Differing GRAE Results

In [1]:
from Helpers.grae_pipeline_helpers import read_all_graeBuild_results
import pandas as pd

In [2]:
grae_results = read_all_graeBuild_results()

In [3]:
grae_results

""
